In [3]:
pip install openai huggingface-hub python-dotenv pandas requests


In [ ]:

# Import required libraries
import os
import openai
import requests
import pandas as pd
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set API keys
openai.api_key = os.getenv("OPENAI_API_KEY")
proxycurl_api_key = os.getenv("PROXYCURL_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
linkedin_client_id = os.getenv("LINKEDIN_CLIENT_ID")
linkedin_client_secret = os.getenv("LINKEDIN_CLIENT_SECRET")
linkedin_redirect_uri = os.getenv("LINKEDIN_REDIRECT_URI")

ModuleNotFoundError: No module named 'openai'

In [ ]:
# Initialize Hugging Face Inference API
hf = InferenceApi(repo_id="distilbert-base-uncased-finetuned-sst-2-english", token=hf_api_key)

# Function to extract text from profile sections
def extract_text_from_sections(profile, sections):
    text = ''
    if 'bio' in sections and profile.get('summary'):
        text += profile['summary'] + ' '
    if 'projects' in sections and profile.get('projects'):
        for project in profile['projects']:
            if project.get('description'):
                text += project['description'] + ' '
    if 'experience' in sections and profile.get('experiences'):
        for experience in profile['experiences']:
            if experience.get('description'):
                text += experience['description'] + ' '
    if 'skills' in sections and profile.get('skills'):
        text += ' '.join(profile['skills']) + ' '
    return text.strip()

In [ ]:
# Function to analyze traits using OpenAI
def analyze_traits(profile):
    try:
        text = extract_text_from_sections(profile, ['bio', 'projects', 'experience'])
        print('Analyzing traits with text:', text)
        prompt = f"Analyze the following text and list the top 7 traits that describe the person professionally: {text}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        traits = response.choices[0].message['content'].split('\n').filter(Boolean)
        print('Generated Traits:', traits)
        return traits
    except Exception as e:
        print('Error analyzing traits:', str(e))
        return []


In [ ]:
# Function to generate ice breakers using OpenAI
def generate_ice_breakers(profile):
    try:
        prompt = f"Generate 3 unique and engaging ice breakers based on this LinkedIn profile. The ice breakers should be professional, related to their work experience or skills, and encourage meaningful conversation. Profile: {profile}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        ice_breakers = response.choices[0].message['content'].split('\n').filter(Boolean)
        print('Generated Ice Breakers:', ice_breakers)
        return ice_breakers
    except Exception as e:
        print('Error generating ice breakers:', str(e))
        return []


In [ ]:
# Function to generate tech skills using OpenAI
def generate_tech_skills(profile):
    try:
        text = extract_text_from_sections(profile, ['bio', 'projects', 'experience'])
        print('Generating tech skills with text:', text)
        prompt = f"Analyze the following text and list the top 10 technical skills that the person possesses: {text}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        tech_skills = response.choices[0].message['content'].split('\n').filter(Boolean)
        print('Generated Tech Skills:', tech_skills)
        return tech_skills
    except Exception as e:
        print('Error generating tech skills:', str(e))
        return []


In [ ]:
# Function to enhance skills with emotion analysis using Hugging Face
def enhance_skills_with_emotion_analysis(skills):
    try:
        enhanced_skills = []
        for skill in skills:
            result = hf(inputs=skill)
            print('Emotion Analysis Result for skill:', skill, result)
            enhanced_skills.append(f"{skill} ({result[0]['label']}: {result[0]['score']:.2f})")
        return enhanced_skills
    except Exception as e:
        print('Error enhancing skills with emotion analysis:', str(e))
        return skills

In [ ]:
# Function to generate job contributions using OpenAI
def generate_job_contributions(profile):
    try:
        prompt = f"Generate 5 significant job contributions based on this LinkedIn profile. The contributions should highlight the impact and achievements in their roles. Profile: {profile}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        job_contributions = response.choices[0].message['content'].split('\n').filter(Boolean)
        print('Generated Job Contributions:', job_contributions)
        return job_contributions
    except Exception as e:
        print('Error generating job contributions:', str(e))
        return []


In [ ]:
# Function to fetch LinkedIn profile using ProxyCurl
def get_linkedin_profile_by_url(linkedin_url):
    try:
        response = requests.get(
            'https://nubela.co/proxycurl/api/v2/linkedin',
            params={'url': linkedin_url},
            headers={'Authorization': f'Bearer {proxycurl_api_key}'}
        )
        return process_proxycurl_data(response.json())
    except Exception as e:
        print('Error fetching LinkedIn data from ProxyCurl:', str(e))
        return {}

In [ ]:
# Function to process ProxyCurl data
def process_proxycurl_data(data):
    return {
        'name': f"{data.get('first_name')} {data.get('last_name')}",
        'headline': data.get('headline'),
        'summary': data.get('summary'),
        'profileUrl': f"https://www.linkedin.com/in/{data.get('public_identifier')}/" if data.get('public_identifier') else None,
        'location': data.get('location'),
        'industry': data.get('industry'),
        'experiences': [{'title': exp.get('title'), 'company': exp.get('company'), 'startDate': exp.get('starts_at'), 'endDate': exp.get('ends_at'), 'description': exp.get('description')} for exp in data.get('experiences', [])],
        'education': [{'school': edu.get('school'), 'degree': edu.get('degree_name'), 'fieldOfStudy': edu.get('field_of_study'), 'startDate': edu.get('starts_at'), 'endDate': edu.get('ends_at')} for edu in data.get('education', [])],
        'skills': data.get('skills', [])
    }

In [ ]:
# Function to search LinkedIn profile using Tavily
def search_linkedin_profile(name):
    try:
        response = requests.get(
            'https://api.tavily.com/search',
            params={
                'api_key': tavily_api_key,
                'query': f'LinkedIn profile of {name}',
                'search_depth': 'advanced',
                'include_domains': 'linkedin.com'
            }
        )
        return process_linkedin_data(response.json())
    except Exception as e:
        print('Error fetching LinkedIn data from Tavily:', str(e))
        return {}


In [ ]:
# Function to process LinkedIn data from Tavily
def process_linkedin_data(tavily_response):
    linkedIn_result = next((result for result in tavily_response.get('results', []) if 'linkedin.com/in/' in result.get('url', '')), None)
    if not linkedIn_result:
        raise ValueError('LinkedIn profile not found')
    return {
        'name': linkedIn_result['title'].split(' | ')[0],
        'headline': linkedIn_result['title'].split(' | ')[1],
        'summary': linkedIn_result['snippet'],
        'profileUrl': linkedIn_result['url']
    }

In [ ]:
# Sample LinkedIn profile data for testing
sample_profile = {
    'summary': 'Experienced data scientist with a demonstrated history of working in the tech industry.',
    'projects': [{'description': 'Developed a machine learning model for predicting customer churn.'}],
    'experiences': [{'description': 'Worked as a data scientist at a leading tech company.'}],
    'skills': ['Python', 'Machine Learning', 'Data Analysis']
}

In [ ]:
# Test functions
traits = analyze_traits(sample_profile)
print("Traits:", traits)

tech_skills = generate_tech_skills(sample_profile)
print("Tech Skills:", tech_skills)

enhanced_tech_skills = enhance_skills_with_emotion_analysis(tech_skills)
print("Enhanced Tech Skills:", enhanced_tech_skills)

job_contributions = generate_job_contributions(sample_profile)
print("Job Contributions:", job_contributions)

# Fetch and display LinkedIn profile data from ProxyCurl
linkedin_profile = get_linkedin_profile_by_url('https://www.linkedin.com/in/sample-profile/')
print("LinkedIn Profile (ProxyCurl):", linkedin_profile)

# Search and display LinkedIn profile data from Tavily
searched_profile = search_linkedin_profile('Sample Name')
print("Searched LinkedIn Profile (Tavily):", searched_profile)